# 모델 버전 관리 및 레지스트리

이 노트북은 MLflow Model Registry를 사용하여 모델의 생명주기를 관리하는 방법을 보여줍니다.

## 학습 내용
- 모델 레지스트리에 모델 등록
- 모델 버전 관리
- 모델 단계 전환 (Staging, Production)
- 모델 메타데이터 관리

In [ ]:
# 필요한 라이브러리 설치
!pip install -q mlflow scikit-learn pandas numpy

In [ ]:
# 라이브러리 임포트
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print(f"MLflow 버전: {mlflow.__version__}")

## 1. 데이터 준비

In [ ]:
# Breast Cancer 데이터셋 로드
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"학습 데이터: {X_train_scaled.shape}")
print(f"테스트 데이터: {X_test_scaled.shape}")

## 2. MLflow Client 설정

In [ ]:
# MLflow 클라이언트 생성
client = MlflowClient()

# 실험 설정
experiment_name = "breast-cancer-classification"
mlflow.set_experiment(experiment_name)

print(f"실험 이름: {experiment_name}")

## 3. 모델 학습 및 등록 (버전 1)

In [ ]:
# 모델 이름 정의
model_name = "breast-cancer-classifier"

# 첫 번째 모델 버전 학습
with mlflow.start_run(run_name="v1-baseline") as run:
    # 파라미터
    params_v1 = {
        'n_estimators': 50,
        'max_depth': 5,
        'random_state': 42
    }
    mlflow.log_params(params_v1)
    mlflow.log_param("version", "1.0")
    
    # 모델 학습
    model_v1 = RandomForestClassifier(**params_v1)
    model_v1.fit(X_train_scaled, y_train)
    
    # 평가
    y_pred = model_v1.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1)
    
    # 모델 저장 및 등록
    mlflow.sklearn.log_model(
        model_v1,
        "model",
        registered_model_name=model_name
    )
    
    run_id_v1 = run.info.run_id
    
    print(f"Version 1 - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")
    print(f"Run ID: {run_id_v1}")

## 4. 개선된 모델 학습 및 등록 (버전 2)

In [ ]:
# 두 번째 모델 버전 학습 (파라미터 개선)
with mlflow.start_run(run_name="v2-improved") as run:
    # 파라미터 개선
    params_v2 = {
        'n_estimators': 100,
        'max_depth': 10,
        'random_state': 42
    }
    mlflow.log_params(params_v2)
    mlflow.log_param("version", "2.0")
    
    # 모델 학습
    model_v2 = RandomForestClassifier(**params_v2)
    model_v2.fit(X_train_scaled, y_train)
    
    # 평가
    y_pred = model_v2.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1)
    
    # 모델 저장 및 등록
    mlflow.sklearn.log_model(
        model_v2,
        "model",
        registered_model_name=model_name
    )
    
    run_id_v2 = run.info.run_id
    
    print(f"Version 2 - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")
    print(f"Run ID: {run_id_v2}")

## 5. 등록된 모델 버전 조회

In [ ]:
# 등록된 모델의 모든 버전 조회
try:
    model_versions = client.search_model_versions(f"name='{model_name}'")
    
    print(f"모델 '{model_name}'의 등록된 버전:")
    for mv in model_versions:
        print(f"\nVersion: {mv.version}")
        print(f"  Stage: {mv.current_stage}")
        print(f"  Run ID: {mv.run_id}")
        print(f"  Status: {mv.status}")
except Exception as e:
    print(f"모델 버전 조회 중 오류: {e}")
    print("참고: Colab 환경에서는 Model Registry 기능이 제한될 수 있습니다.")

## 6. 모델 단계 전환

In [ ]:
# 버전 1을 Staging으로 전환
try:
    client.transition_model_version_stage(
        name=model_name,
        version=1,
        stage="Staging"
    )
    print("Version 1이 Staging 단계로 전환되었습니다.")
except Exception as e:
    print(f"단계 전환 중 오류: {e}")
    print("참고: Colab 환경에서는 Model Registry 기능이 제한될 수 있습니다.")

In [ ]:
# 버전 2를 Production으로 전환 (성능이 더 좋다고 판단)
try:
    client.transition_model_version_stage(
        name=model_name,
        version=2,
        stage="Production"
    )
    print("Version 2가 Production 단계로 전환되었습니다.")
except Exception as e:
    print(f"단계 전환 중 오류: {e}")
    print("참고: Colab 환경에서는 Model Registry 기능이 제한될 수 있습니다.")

## 7. 특정 단계의 모델 로드

In [ ]:
# Production 단계의 모델 로드
try:
    model_uri = f"models:/{model_name}/Production"
    production_model = mlflow.sklearn.load_model(model_uri)
    
    # 예측
    predictions = production_model.predict(X_test_scaled[:5])
    print("Production 모델의 예측:")
    for i, pred in enumerate(predictions):
        result = "Malignant" if pred == 0 else "Benign"
        print(f"샘플 {i+1}: {result}")
except Exception as e:
    print(f"모델 로드 중 오류: {e}")
    print("대신 Run ID를 사용하여 모델을 로드합니다.")
    
    # Run ID로 모델 로드
    model_uri = f"runs:/{run_id_v2}/model"
    production_model = mlflow.sklearn.load_model(model_uri)
    
    predictions = production_model.predict(X_test_scaled[:5])
    print("\n최신 모델의 예측:")
    for i, pred in enumerate(predictions):
        result = "Malignant" if pred == 0 else "Benign"
        print(f"샘플 {i+1}: {result}")

## 8. 모델 버전에 설명 추가

In [ ]:
# 모델 버전에 설명 추가
try:
    client.update_model_version(
        name=model_name,
        version=1,
        description="Baseline model with n_estimators=50, max_depth=5"
    )
    
    client.update_model_version(
        name=model_name,
        version=2,
        description="Improved model with n_estimators=100, max_depth=10. Better accuracy."
    )
    
    print("모델 버전 설명이 업데이트되었습니다.")
except Exception as e:
    print(f"설명 업데이트 중 오류: {e}")

## 9. 모델 버전 비교

In [ ]:
# 실험의 모든 실행 조회
experiment = mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.accuracy DESC"]
)

# 주요 메트릭 비교
comparison_cols = ['run_id', 'params.version', 'metrics.accuracy', 
                   'metrics.f1_score', 'params.n_estimators', 'params.max_depth']
display_cols = [col for col in comparison_cols if col in runs.columns]

print("\n모델 버전 비교:")
print(runs[display_cols])

## 10. 모델 아카이브

In [ ]:
# 오래된 버전을 Archived 단계로 전환
try:
    client.transition_model_version_stage(
        name=model_name,
        version=1,
        stage="Archived"
    )
    print("Version 1이 Archived 단계로 전환되었습니다.")
except Exception as e:
    print(f"아카이브 중 오류: {e}")

## 요약

이 노트북에서는 MLflow Model Registry를 사용하여:
1. ✅ 모델을 레지스트리에 등록
2. ✅ 여러 모델 버전 생성 및 관리
3. ✅ 모델 단계 전환 (None → Staging → Production → Archived)
4. ✅ 특정 단계의 모델 로드 및 사용
5. ✅ 모델 버전에 메타데이터 추가
6. ✅ 모델 버전 비교

**참고**: Colab 환경에서는 일부 Model Registry 기능이 제한될 수 있습니다. 
완전한 기능을 사용하려면 MLflow Tracking Server를 설정해야 합니다.

다음 단계: 모델 배포 및 서빙